In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
import sklearn.model_selection as model_selection
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import spacy
# from sklearn.utils import shuffle
# import stop_words
TRAIN_LABELS = 'participants/train/labels/labels.csv'
TRAIN_TEXT = 'participants/train/extracted_data/extract_combined.csv'
TEST_TEXT = 'participants/test/extracted_data/extract_combined.csv'
TRAIN_ORG_TEXT = 'participants/get_only_ORG.csv'

In [10]:
train_labels_df = pd.read_csv(TRAIN_LABELS, usecols=['document_name','is_fitara'])
train_text_df = pd.read_csv(TRAIN_TEXT)
test_df = pd.read_csv(TEST_TEXT)


In [11]:
train_labels_df.is_fitara = train_labels_df.is_fitara.eq('Yes').mul(1)

train_df = pd.merge(
    train_labels_df, 
    train_text_df, 
    on='document_name', 
    how='inner'
)
train_df.head()

,document_name,is_fitara,text
0,04-42RFP.pdf,0,\n \nOMB No. 0990-0115 \n \n Electronic Requ...
1,1_-_Brand_name_JOFOC_1782798.pdf,0,\n \nJustification for Other than Full and Op...
2,1_-_Brand_Name_JOFOC_for_FBO.docx,0,Justification for Other than Full and Open Com...
3,1_-_Brand_Name_Only.docx,1,Justification for Other than Full and Open Com...
4,1_-_JOFOC_-_FSSI_Limited_Source.doc,1,Attachment C\r\r\rHHS Template and Instruction...


In [23]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        return: modified initial string
    """
    text = text.lower()  # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)  # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text)  # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing.
    text = text.replace('x', '')
    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)  # remove stopwors from text
    return text

nlp_spacy = spacy.load('en')
count = 0
def data_pre_processing(text):

    global count
#     print("INFO - processing - " + text[:4])
#     text = text[:200]
#     print(text)
    texts = []
    doc = nlp_spacy(text, disable=['parser', 'ner'])
    tokens = [token for token in doc if not token.is_stop]
    tokens = [token for token in tokens if not token.is_punct]
    tokens = [token.lemma_.lower().strip() for token in tokens if token.lemma_ != '-PRON-']
    tokens = ' '.join(tokens)
    count += 1
    print(count,len(tokens))
    texts.append(tokens)
    return ' '.join(texts)
#     return pd.Series([texts, len(tokens)])

# result = train_df['text'].apply(lambda x: data_pre_processing(x))
# train_df['processed_text'], train_df['wc'] = result[0], result[1]

In [24]:
# nltk.download('stopwords')

In [25]:
train_df['text'] = train_df['text'].apply(data_pre_processing)

1 138774
2 4441
3 3039
4 2996
5 8370
6 3356
7 3486
8 41313
9 36068
10 51268
11 51807
12 10551
13 2481
14 2618
15 2821
16 175554
17 11308
18 5655
19 5644
20 3240
21 87290
22 185263
23 3570
24 5573
25 1448
26 4381
27 4195
28 11647
29 3930
30 3749
31 14137
32 10255
33 61680
34 2950
35 33109
36 13254
37 476
38 7500
39 193
40 252
41 3815
42 9057
43 16072
44 7042
45 1360
46 5889
47 2171
48 5018
49 2351
50 6230
51 3696
52 17004
53 5502
54 480
55 12398
56 12390
57 1910
58 12313
59 9236
60 2852
61 12388
62 12392
63 3426
64 865
65 1898
66 1228
67 12164
68 1788
69 3928
70 113740
71 3254
72 3115
73 9279
74 57858
75 53440
76 6138
77 2605
78 6875
79 2979
80 2452
81 1684
82 23729
83 1423
84 756
85 1792
86 3158
87 44044
88 1181
89 15053
90 1883
91 1759
92 1892
93 67335
94 952
95 18325
96 8652
97 7799
98 25786
99 25764
100 12050
101 14049
102 12490
103 12509
104 12509
105 11588
106 11589
107 13317
108 13317
109 12308
110 12308
111 12308
112 15798
113 11533
114 11533
115 11531
116 11535
117 11533
118 11

868 13311
869 2043
870 3859
871 3168
872 3766
873 3695
874 4847
875 5270
876 3575
877 2007
878 3827
879 2205
880 36868
881 1100
882 6160
883 53088
884 8694
885 3400
886 3459
887 2897
888 64175
889 67112
890 58485
891 6440
892 16755
893 1272
894 6793
895 4016
896 1696
897 3046
898 2867
899 1678
900 57453
901 22936
902 1158
903 10817
904 1436
905 24613
906 14463
907 16770
908 16770
909 16770
910 1487
911 1495
912 3117
913 2628
914 1117
915 4476
916 2339
917 3697
918 1269
919 4239
920 5249
921 4968
922 9528
923 14556
924 2590
925 2615
926 2759
927 25360
928 14546
929 1865
930 6537
931 226834
932 11857
933 36144
934 1773
935 53440
936 21851
937 5271
938 25823
939 12386
940 2167
941 1323
942 4807
943 3978
944 8301
945 23510
946 23510
947 23510
948 5797
949 29649
950 22688
951 22708
952 12144
953 7138
954 5073
955 2162


In [27]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(train_df.text, train_df.is_fitara, test_size=0.3)

In [94]:
tfidf_vect_ngram = TfidfVectorizer( max_df=0.7, analyzer='word', ngram_range=(2,3), token_pattern='\w{1,}' ,max_features=250000)
#stop_words='english',
#, token_pattern=r'\w{1,}'
tfidf_vect_ngram.fit(train_x)
ngram_train = tfidf_vect_ngram.transform(train_x.values)
ngram_test = tfidf_vect_ngram.transform(test_x.values)

In [95]:
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
# Loss 4.45
nb_classifier2 = SGDClassifier(max_iter=100, verbose=0)#class_weight={ 0:0.3, 1:0.7 })
# for i in range(3):
nb_classifier2.fit(ngram_train, train_y)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=100,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [96]:
# Loss 6.98
nb_classifier2 = MultinomialNB(class_prior=[.5, .5])
nb_classifier2.fit(ngram_train, train_y)
ngram_pred = nb_classifier2.predict(ngram_test)
ngram_score = metrics.accuracy_score(test_y, ngram_pred)

In [97]:

ngram_pred = nb_classifier2.predict(ngram_test)
ngram_score = metrics.accuracy_score(test_y, ngram_pred)
print('============= NGRAM VECTORIZER ================')
print('Loss = ',metrics.log_loss(test_y, ngram_pred))
print('Score = ',ngram_score)
print('CM = \n',metrics.confusion_matrix(test_y, ngram_pred, labels=[0, 1]))

============= NGRAM VECTORIZER ================
Loss =  5.8968893370296005
Score =  0.8292682926829268
CM = 
 [[204   9]
 [ 40  34]]


### Prepare for submission

In [103]:
ngram_text_vectors = tfidf_vect_ngram.transform(test_df.text.values)
ngram_validate_pred = nb_classifier2.predict_proba(ngram_text_vectors)


predictions_df = pd.DataFrame()
predictions_df['document_name'] = test_df.document_name
predictions_df['pred_fitara'] = np.round(ngram_validate_pred[:,1], decimals= 4)
# predictions_df.to_csv('participants/predictions.csv', sep=',', index=False)

In [106]:
predictions_df.to_csv('participants/predictions(11306).csv', sep=',', index=False)

### Spacy

In [15]:
import spacy
nlp = spacy.load('en')


In [6]:
doc = nlp(train_df.text[3])

In [7]:
labels = set()
for ent in doc.ents:
    labels.add(ent.label_)
    if ent.label_ == 'ORG':
        print(ent.text, ent.label_)

Norovirus Therapeutics ORG
Vaccines Total ORG
ETA ORG
Noroviruses ORG
KingFisher Flex Purification System ORG
KingFisher Flex Purification System ORG
Manufacturer is Life Technologies Corporation ORG
Thermo Fisher Scientific ORG
Grand ORG
RNA ORG
USC ORG
FSS ORG
NIH ORG
Contracting Officer ORG
USC ORG
Unusual & ORG
USC ORG
USC ORG
USC ORG
USC ORG
USC ORG
Life Technologies ORG


In [27]:
doc2 = nlp(train_df.text[15])

In [5]:
for ent in doc2.ents:
    if ent.label_ == 'ORG':
        print(ent.text, ent.label_)

NameError: name 'doc2' is not defined

In [16]:
count = 0
def get_only_ORG(text):
    global count
    doc = nlp(text)
    ORGS = ""
    for ent in doc.ents:
        if ent.label_ == 'ORG':
            ORGS += " "
            ORGS += ent.text
    count += 1
    if len(ORGS) == 0:
        print("Not found")
    print(count)
    return ORGS

# count = 0
train_df['text'] = train_df['text'].apply(get_only_ORG)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
Not found
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
27

In [20]:
train_df.to_csv('participants/get_only_ORG.csv', sep='\t', index=False)

In [22]:
d = pd.read_csv('participants/get_only_ORG.csv', sep = '\t')

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.
